In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for dataset
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec30/train', transform=transform)
val_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec30/val', transform=transform)
test_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec30/test', transform=transform)

# Class mapping
class_map = {i: train_dataset.classes[i] for i in range(len(train_dataset.classes))}

In [7]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

3201
1058
1058


In [8]:
import torch.nn as nn

# Define ResNet model
class ResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet18.conv1(x)
        x = self.resnet18.bn1(x)
        x = self.resnet18.relu(x)
        x = self.resnet18.maxpool(x)

        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)

        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def train(model, optimizer, criterion, labeled_loader):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    total = 0
    for images, labels in labeled_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_acc += (predicted == labels).sum().item()
    train_loss /= len(labeled_loader)
    train_acc /= total
    return train_loss, train_acc

In [9]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=4, verbose=False, delta=0, path='checkpoint.pt', min_epoch = 0, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.epoch = 0
        self.min_epoch = min_epoch
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss
        self.epoch +=1
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            
            if self.epoch < self.min_epoch:
                self.counter = 0
                self.trace_func("Not enough epoch")
            else:
                self.counter += 1
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [10]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

test_loss_unc = []
test_acc_unc = []
test_auc_score_unc = []
test_roc_auc_unc = []
test_precision_unc = []
test_recall_unc = []
test_f1_unc = []

df_unc30 = pd.DataFrame(columns = train_dataset.classes)

num_initial_samples = 100
batch_size = 128

# Define initial labeled dataset
labeled_indices_unc = torch.randperm(len(train_dataset))[:num_initial_samples]
labeled_dataset_unc = torch.utils.data.Subset(train_dataset, labeled_indices_unc)

unlabeled_indices_unc = torch.arange(len(train_dataset))[~torch.eq(torch.arange(len(train_dataset)).unsqueeze(1), labeled_indices_unc).any(1)]
unlabeled_dataset_unc = torch.utils.data.Subset(train_dataset, unlabeled_indices_unc)

# Define data loaders
labeled_loader_unc = DataLoader(labeled_dataset_unc, batch_size=128, shuffle=True)
unlabeled_loader_unc = DataLoader(unlabeled_dataset_unc, batch_size=128, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


step = 1
# Active learning loop
while(len(unlabeled_dataset_unc) > 0):
    print("Step number:", step)

    model_unc = ResNet18(num_classes=30).to(device)
    early_stopping_unc = EarlyStopping(patience = 20, verbose = True, min_epoch = 0)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model_unc.parameters(), lr=0.0001)

    for epoch in range(200):
        # Train model on labeled dataset
        train_loss, train_acc = train(model_unc, optimizer, criterion, labeled_loader_unc)
        print(f"Step : {step}, Epoch : {epoch+1} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")    

    # Evaluate model on val dataset
        model_unc.eval()
        with torch.no_grad():
            val_loss = 0.0
            val_acc = 0.0
            total = 0
            y_true = []
            y_pred = []
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_unc(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                val_acc += (predicted == labels).sum().item()
                y_true.extend(labels.tolist())
                y_pred.extend(predicted.tolist())
            val_loss /= len(val_loader)
            val_acc /= total

            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'macro')

            # Convert multi-class labels to binary labels (OVR)
            lb = LabelBinarizer()
            lb.fit(y_true)
            y_true_bin = lb.transform(y_true)
            y_pred_bin = lb.transform(y_pred)
            # Compute AUC score and ROC curve
            fpr, tpr, thresholds = roc_curve(y_true_bin.ravel(), y_pred_bin.ravel())
            auc_score = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
            roc_auc = auc(fpr, tpr)
            
            # Early Stopping Condition
            early_stopping_unc(val_loss, model_unc)
            
            if early_stopping_unc.early_stop:
                print("Early stopping")
                print(f"Val Precision: {precision.item():.4f}, Val Recall: {recall.item():.4f}, Val F1 Score: {f1.item():.4f}")
                break
            
            print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")            
            print(f"AUC Score: {auc_score:.4f}")
            print("")
            
        model_unc.load_state_dict(torch.load('checkpoint.pt'))

    
    # Evaluate model on Test Dataset
    model_unc.load_state_dict(torch.load('checkpoint.pt'))
    model_unc.eval()
    with torch.no_grad():
        test_loss = 0.0
        test_acc = 0.0
        total = 0
        y_true = []
        y_pred = []
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_unc(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            test_acc += (predicted == labels).sum().item()
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
        test_loss /= len(test_loader)
        test_acc /= total
        
        # Convert multi-class labels to binary labels (OVR)
        lb = LabelBinarizer()
        lb.fit(y_true)
        y_true_bin = lb.transform(y_true)
        y_pred_bin = lb.transform(y_pred)
        # Compute AUC score and ROC curve
        fpr, tpr, thresholds = roc_curve(y_true_bin.ravel(), y_pred_bin.ravel())
        auc_score_unc = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
        roc_auc_unc = auc(fpr, tpr)

        precision_unc, recall_unc, f1_unc, _ = precision_recall_fscore_support(y_true, y_pred, average = 'macro')
        test_loss_unc.append(test_loss)
        test_acc_unc.append(test_acc)
        test_auc_score_unc.append(auc_score_unc)
        test_roc_auc_unc.append(roc_auc_unc)
        test_precision_unc.append(precision_unc)
        test_recall_unc.append(recall_unc)
        test_f1_unc.append(f1_unc)

        
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
        print(f"AUC Score: {auc_score_unc:.4f}")
    

    # Iteration end condition
    if len(labeled_dataset_unc) > int(len(train_dataset)/2):
        break
    
    # Make model predictions on unlabeled dataset
    model_unc.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in unlabeled_loader_unc:
            images = images.to(device)
            output = model_unc(images)
            predictions.append(output)
    predictions = torch.cat(predictions, dim=0)

    # Select samples to label using entropy-based method
    entropy = -torch.sum(torch.softmax(predictions, dim=1) * torch.log(torch.softmax(predictions, dim=1)), dim=1)
    _, idx = torch.topk(entropy, k = 20) # select top 20


    # Count selected classes
    class_counts = {class_name: 0 for class_name in train_dataset.classes}

    for i in idx:
        class_index = torch.argmax(predictions[i]).item()
        class_name = train_dataset.classes[class_index]
        class_counts[class_name] += 1

    new_row = {class_name: class_counts[class_name] for class_name in train_dataset.classes}
    df_unc30 = df_unc30.append(new_row, ignore_index=True)

    # print("data frame update")
    # print(df_unc30)
    

    labeled_indices_unc = torch.cat([labeled_indices_unc, unlabeled_indices_unc[idx]])
    unlabeled_indices_unc = unlabeled_indices_unc[~torch.eq(unlabeled_indices_unc.unsqueeze(1), unlabeled_indices_unc[idx]).any(1)]

    labeled_dataset_unc = torch.utils.data.Subset(train_dataset, labeled_indices_unc)
    unlabeled_dataset_unc = torch.utils.data.Subset(train_dataset, unlabeled_indices_unc)
    print("Updated Length of labeled dataset : ",len(labeled_dataset_unc))
    print("Length of unlabeled dataset : ",len(unlabeled_dataset_unc))
    print("-----------------------------------------------------------")

    # Update labeled and unlabeled data loaders
    labeled_loader_unc = DataLoader(labeled_dataset_unc, batch_size=batch_size, shuffle=True)
    unlabeled_loader_unc = DataLoader(unlabeled_dataset_unc, batch_size=batch_size, shuffle=True)

    step+=1

Step number: 1
Step : 1, Epoch : 1 - Train Loss: 3.6588, Train Accuracy: 0.0100
Validation loss decreased (inf --> 3.009545).  Saving model ...
Val Loss: 3.0095, Val Accuracy: 0.2769
AUC Score: 0.5863

Step : 1, Epoch : 2 - Train Loss: 2.4372, Train Accuracy: 0.5300
Validation loss decreased (3.009545 --> 2.574447).  Saving model ...
Val Loss: 2.5744, Val Accuracy: 0.4348
AUC Score: 0.6543

Step : 1, Epoch : 3 - Train Loss: 1.6625, Train Accuracy: 0.7300
Validation loss decreased (2.574447 --> 2.201735).  Saving model ...
Val Loss: 2.2017, Val Accuracy: 0.5227
AUC Score: 0.6806

Step : 1, Epoch : 4 - Train Loss: 1.2141, Train Accuracy: 0.8300
Validation loss decreased (2.201735 --> 1.906875).  Saving model ...
Val Loss: 1.9069, Val Accuracy: 0.5860
AUC Score: 0.7087

Step : 1, Epoch : 5 - Train Loss: 0.9215, Train Accuracy: 0.8500
Validation loss decreased (1.906875 --> 1.675476).  Saving model ...
Val Loss: 1.6755, Val Accuracy: 0.6531
AUC Score: 0.7331

Step : 1, Epoch : 6 - Train Lo

In [11]:
print(test_loss_unc)
print(test_acc_unc)
print(test_auc_score_unc)
print(test_roc_auc_unc)
print(test_precision_unc)
print(test_recall_unc)
print(test_f1_unc)

[0.9393629497951932, 0.8328516317738427, 0.7284879667891396, 0.6789778255754046, 0.6897739751471413, 0.6962578478786681, 0.6811840472122034, 0.6519823148846626, 0.5999514245324664, 0.534266520705488, 0.5418507729967436, 0.5279263903697332, 0.4974880384074317, 0.5046078869038157, 0.4582109947999318, 0.47980952511231106, 0.45747513986296123, 0.4616990697880586, 0.4246518752641148, 0.44442709121439194, 0.44728732026285595, 0.46110747589005363, 0.43796561658382416, 0.395307538823949, 0.4117661536567741, 0.41163550855384934, 0.3942290370663007, 0.39509087469842696, 0.42142156325280666, 0.3757382531960805, 0.37117409333586693, 0.40375884663727546, 0.4093594070937898, 0.39057164101137054, 0.38753819051716065, 0.3893074455360572, 0.35942289440168274, 0.3650674455695682, 0.34728924019469154, 0.3471858782900704, 0.3517610977093379, 0.310004082818826, 0.3178405923147996, 0.33104030829336906, 0.3258756349484126, 0.3187323270572556, 0.34509225231077933, 0.37431462440225816, 0.3203077101045185, 0.28

In [12]:
df_unc30

,bottle_anomaly,bottle_good,cable_anomaly,cable_good,capsule_anomaly,capsule_good,carpet_anomaly,carpet_good,grid_anomaly,grid_good,...,tile_anomaly,tile_good,toothbrush_anomaly,toothbrush_good,transistor_anomaly,transistor_good,wood_anomaly,wood_good,zipper_anomaly,zipper_good
0,0,0,0,0,0,0,0,0,0,0,...,3,2,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,6,...,0,0,9,0,1,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,2,0,15,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,19,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1,1,2,1,0,0,1,0,0,0,...,0,0,1,1,3,1,0,1,0,0
72,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,3,1,0,0,0,0
73,3,1,0,0,0,1,0,0,0,1,...,0,0,1,2,0,0,0,1,0,0
74,2,1,1,2,1,0,1,0,0,0,...,0,0,0,2,0,1,0,0,0,0


In [13]:
df_unc30.to_csv('/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/active_learning_30_classes/df_unc30.csv')

: 